In [32]:
import os
import json
import pandas as pd
import numpy as np


In [33]:
# Chemin complet vers le csv servant de source.
source_csv = '../data/metrics/merge_raw_metrics_dataset.csv'

# Chemin de stockage des csv
path = "../data/metrics/"

## <center>**Import du Dataset originel**<center>

In [34]:
# Création Dataframe
df = pd.read_csv(source_csv, index_col=0).reset_index(drop=True).dropna(axis=1, how="all")

C:\Users\Arthur\AppData\Local\Temp\ipykernel_4612\3036002175.py:2: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_csv, index_col=0).reset_index(drop=True).dropna(axis=1, how="all")


In [35]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546276 entries, 0 to 3546275
Data columns (total 16 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         int64  
 1   name_modules               object 
 2   type_modules               object 
 3   generation_modules         object 
 4   name_counters_modules      object 
 5   value_counters_modules     int64  
 6   name_connected_operators   object 
 7   level_connected_operators  object 
 8   source_events              object 
 9   message_events             object 
 10  timestamp_events           object 
 11  criticality_events         object 
 12  identification_events      object 
 13  created_at                 object 
 14  varnishLevelsTargetvolume  float64
 15  varnishLevelsTotalvolume   int64  
dtypes: float64(1), int64(3), object(12)
memory usage: 432.9+ MB


,id,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,source_events,message_events,timestamp_events,criticality_events,identification_events,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,4169748,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,NaN,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
1,4169748,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,NaN,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
2,4169748,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,NaN,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
3,4169749,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15T05:55:23.462Z,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
4,4169749,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15T05:55:23.462Z,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000


## <center>**Préparation du Dataset**<center>

### I - Imputation des données manquantes et suppression des colonnes inutiles 

In [36]:
# suppression des colonnes inutiles ou en doublon
df.drop(["id", "message_events"], axis = 1)

,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,source_events,timestamp_events,criticality_events,identification_events,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
1,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
2,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,NaN,NaN,NaN,NaN,2022-04-15 05:55:06.678000,36192.322612,100000
3,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,PLC,2022-04-15T05:55:23.462Z,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
4,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,PLC,2022-04-15T05:55:23.462Z,INFO,391,2022-04-15 05:55:06.829000,36192.322612,100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546271,iFoil L,iFoil,Gen. 2,Total Pages Counter,55387,Viktor,Operator,NaN,NaN,NaN,NaN,2022-12-12 08:21:28.092000,88999.004996,100000
3546272,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,76433,Viktor,Operator,NaN,NaN,NaN,NaN,2022-12-12 08:21:28.092000,88999.004996,100000
3546273,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,2547042,Viktor,Operator,NaN,NaN,NaN,NaN,2022-12-12 08:21:38.102000,88999.004996,100000
3546274,iFoil L,iFoil,Gen. 2,Total Pages Counter,55387,Viktor,Operator,NaN,NaN,NaN,NaN,2022-12-12 08:21:38.102000,88999.004996,100000


In [37]:
# Remplacement les données manquants dans la colonne "timestamp_events" par les timestamp de la colonne "created_at"
df.timestamp_events  = df.timestamp_events.fillna(df["created_at"])
# Suppression de la colonne "created_at" devenue superflue
df = df.drop(["created_at"], axis= 1)
# Renommage de la colonne "timestamp_events"
df.rename(columns={"timestamp_events" : "timestamp"})


,id,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,source_events,message_events,timestamp,criticality_events,identification_events,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,4169748,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,NaN,NaN,2022-04-15 05:55:06.678000,NaN,NaN,36192.322612,100000
1,4169748,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,NaN,NaN,2022-04-15 05:55:06.678000,NaN,NaN,36192.322612,100000
2,4169748,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,NaN,NaN,2022-04-15 05:55:06.678000,NaN,NaN,36192.322612,100000
3,4169749,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15T05:55:23.462Z,INFO,391,36192.322612,100000
4,4169749,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,PLC,JV-Ti non prêt : impression impossible,2022-04-15T05:55:23.462Z,INFO,391,36192.322612,100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546271,10909314,iFoil L,iFoil,Gen. 2,Total Pages Counter,55387,Viktor,Operator,NaN,NaN,2022-12-12 08:21:28.092000,NaN,NaN,88999.004996,100000
3546272,10909314,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,76433,Viktor,Operator,NaN,NaN,2022-12-12 08:21:28.092000,NaN,NaN,88999.004996,100000
3546273,10909319,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,2547042,Viktor,Operator,NaN,NaN,2022-12-12 08:21:38.102000,NaN,NaN,88999.004996,100000
3546274,10909319,iFoil L,iFoil,Gen. 2,Total Pages Counter,55387,Viktor,Operator,NaN,NaN,2022-12-12 08:21:38.102000,NaN,NaN,88999.004996,100000
